## Powerco

Powerco has three electricity power plants that supply the needs of four cities.
- The supply capacity of each power plant and the demand in each city (in million
KWH/hour) are given in the table below.

| From/to | City 1 | City 2 | City 3 | City 4 | Supply |
| --- | --- | --- | --- | --- | --- |
| Plant 1 | $8 | $6 | $10 | $9 | 35 |
| Plant 2 | $9 | $12 | $13 | $7 | 50 |
| Plant 3 | $14 | $9 | $16 | $5 | 40 |
| Demand | 45 | 20 | 30 | 30 |

- The cost of sending 1 million KWH of electricity from each plant to each city is
also given the table.

**Question:** How does Powerco make its distribution plan to minimize the total cost
while satisfying all cities’ demands?

This time, we will model this as a Minimum Cost Network Flow Problem, that is, our model will take the form:

\begin{align*}
\underset{x \in \mathbb{R}^{|\mathcal{E}|}}{\min} \quad & c^Tx \\
\text{s.t.} \quad \ \ &Ax = b \\
&p \leq x \leq q,
\end{align*}

where $A$ is the incidence matrix of the graph.


### Data

In [1]:
numplants = 3
numcities = 4
numedges = numplants*numcities

#Define the incidence matrix A for our fully connected network
#Edges go from the power plants (first 3 rows) to the cities (last 4 rows)
#Edge 1 is column 1, edge 2 is column 2, etc.
A = [1  1  1  1  0  0  0  0  0  0  0  0
     0  0  0  0  1  1  1  1  0  0  0  0
     0  0  0  0  0  0  0  0  1  1  1  1
    -1  0  0  0 -1  0  0  0 -1  0  0  0
     0 -1  0  0  0 -1  0  0  0 -1  0  0
     0  0 -1  0  0  0 -1  0  0  0 -1  0
     0  0  0 -1  0  0  0 -1  0  0  0 -1]

#Supply (first 3 entries) and demand (last 4 entries)
b = [35, 50, 40, -45, -20, -30, -30]

#Same as before, vectorized
costs = [8 6 10 9 9 12 13 7 14 9 16 5];

A

7×12 Matrix{Int64}:
  1   1   1   1   0   0   0   0   0   0   0   0
  0   0   0   0   1   1   1   1   0   0   0   0
  0   0   0   0   0   0   0   0   1   1   1   1
 -1   0   0   0  -1   0   0   0  -1   0   0   0
  0  -1   0   0   0  -1   0   0   0  -1   0   0
  0   0  -1   0   0   0  -1   0   0   0  -1   0
  0   0   0  -1   0   0   0  -1   0   0   0  -1

### Model

In [2]:
using JuMP, HiGHS

power2 = Model(HiGHS.Optimizer)

@variable(power2, x[1:numedges] >= 0)
@constraint(power2, supplyanddemand, A*x .== b)

@objective(power2, Min, sum(costs[i]*x[i] for i in 1:numedges))

print(power2)

In [3]:
optimize!(power2)

Presolving model
7 rows, 12 cols, 24 nonzeros
6 rows, 12 cols, 20 nonzeros
Presolve : Reductions: rows 6(-1); columns 12(-0); elements 20(-4)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 6(215) 0s
          7     1.0200000000e+03 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 7
Objective value     :  1.0200000000e+03
HiGHS run time      :          0.00


In [4]:
@show objective_value(power2)
@show value.(x)

objective_value(power2) = 1020.0
value.(x) = [0.0, 10.0, 25.0, 0.0, 45.0, 0.0, 5.0, 0.0, 0.0, 10.0, 0.0, 30.0]


12-element Vector{Float64}:
  0.0
 10.0
 25.0
  0.0
 45.0
  0.0
  5.0
  0.0
  0.0
 10.0
  0.0
 30.0